# **Mounting data from drive**

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
!ls "/content/drive/My Drive"


'Colab Notebooks'   Data  'Project Data'


# **Data path**

In [4]:
import os
train_dir = os.path.join('/content/drive/My Drive/Project Data/')

In [5]:
train_dir

'/content/drive/My Drive/Project Data/'

In [6]:
train_dir_names = os.listdir(train_dir)
print(train_dir_names[:10])

['meteo_feature_full.npy', 'site_label.npy', 'time_stamp_full.npy', 'y_PM25_full.txt', 'X_image_full']


In [85]:
train_x_dir=os.path.join('/content/drive/My Drive/Project Data/X_image_full/')
train_dir_names = os.listdir(train_x_dir)
print(len(train_dir_names))
print(train_dir_names[:10])

13022
['X_image_12031_20190505_26.png', 'X_image_12020_20181017_26.png', 'X_image_12024_20190325_26.png', 'X_image_12023_20181217_26.png', 'X_image_12019_20190415_26.png', 'X_image_12021_20190220_26.png', 'X_image_12026_20190629_26.png', 'X_image_12028_20190529_26.png', 'X_image_12027_20181221_26.png', 'X_image_12045_20181110_26.png']


# **Importing libraries**

In [9]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

from itertools import compress
from scipy import stats
import copy
import cv2
import math
import pandas as pd

from sklearn.neighbors import LocalOutlierFactor
from scipy.interpolate import interp1d
from datetime import datetime
from datetime import timedelta
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import os
import requests
from requests.auth import HTTPBasicAuth
import json
from multiprocessing.dummy import Pool as ThreadPool
import time
import shlex, subprocess
import urllib.request
import webbrowser
import copy
import imageio
from os import listdir
from os.path import isfile, join
import tensorflow as tf
import random
import keras
from keras import applications
from keras.utils import multi_gpu_model
from keras.optimizers import RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import  image
from keras import optimizers
from keras.models import Sequential, Model, load_model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, GlobalMaxPooling2D, Conv2D, MaxPooling2D, Activation
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, Callback,LearningRateScheduler, TensorBoard, EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input


# **Sorting the images**

In [11]:
my_X_images_folder = '/content/drive/My Drive/Project Data/X_image_full/'
image_files = [f for f in listdir(my_X_images_folder) if isfile(join(my_X_images_folder, f)) and '.DS_Store' not in f ]
image_files_sorted = sorted(image_files, key = lambda x: int(x.split('.')[0].split('_')[2]))

image_files_sorted

['X_image_0_20170409_0.png',
 'X_image_1_20170625_0.png',
 'X_image_2_20170825_0.png',
 'X_image_3_20170426_0.png',
 'X_image_4_20170620_0.png',
 'X_image_5_20170609_0.png',
 'X_image_6_20170217_0.png',
 'X_image_7_20170615_0.png',
 'X_image_8_20170213_0.png',
 'X_image_9_20170920_0.png',
 'X_image_10_20170912_0.png',
 'X_image_11_20170428_0.png',
 'X_image_12_20170912_0.png',
 'X_image_13_20170626_0.png',
 'X_image_14_20170211_0.png',
 'X_image_15_20170418_0.png',
 'X_image_16_20170908_0.png',
 'X_image_17_20170807_0.png',
 'X_image_18_20170604_0.png',
 'X_image_19_20170224_0.png',
 'X_image_20_20170409_0.png',
 'X_image_21_20170906_0.png',
 'X_image_22_20170627_0.png',
 'X_image_23_20170618_0.png',
 'X_image_24_20170928_0.png',
 'X_image_25_20170224_0.png',
 'X_image_26_20170109_0.png',
 'X_image_27_20170504_0.png',
 'X_image_28_20170711_0.png',
 'X_image_29_20170428_0.png',
 'X_image_30_20170917_0.png',
 'X_image_31_20170906_0.png',
 'X_image_32_20170919_0.png',
 'X_image_33_2017091

# **Data Preprocessing**

## **Preprocessing the Training Data**

In [146]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=30,
        zoom_range=0.5,
        horizontal_flip=True,
        vertical_flip=True,
        width_shift_range=0.1,
        height_shift_range=0.1,
        )


training_set = train_datagen.flow_from_directory(
        "/content/drive/My Drive/Project Data/",
        target_size=(224,224),
        
      
        
        )

Found 13022 images belonging to 1 classes.


In [141]:
y_PM25 = []
with open("/content/drive/My Drive/Project Data/y_PM25_full.txt", "r") as f:
    for line in f:
        y_PM25.append(np.float32(float(line.strip())))


      

In [115]:
(y_PM25[1])

27.75

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_dir_names, y_PM25, test_size=0.33, random_state=42)

In [56]:
type(X_train)

list

In [61]:
np.array(X_train[0])

array('X_image_3952_20180903_9.png', dtype='<U27')

In [60]:
array

array(['X_image_3952_20180903_9.png', 'X_image_8435_20180523_33.png',
       'X_image_1288_20170426_17.png', ...,
       'X_image_7405_20180131_27.png', 'X_image_12884_20190316_33.png',
       'X_image_5293_20180324_16.png'], dtype='<U29')

In [66]:
k=[str(k) for k in X_train]

In [147]:
dataset_train = np.ndarray(shape=(len(X_train), 3, 224, 224), dtype=np.float32)

In [101]:
dataset_train.shape

(8724, 3, 67, 67)

In [148]:
dataset_test = np.ndarray(shape=(len(X_test), 3, 224, 224), dtype=np.float32)

In [116]:
type(dataset_train[0])

numpy.ndarray

In [103]:
dataset_test.shape

(4298, 3, 67, 67)

In [120]:
y_train=np.array(y_train)

In [136]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(67, 67, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution

    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class
    tf.keras.layers.Dense(1, activation='relu')
])


In [137]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 65, 65, 16)        448       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 32, 32, 16)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 16384)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 512)               8389120   
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 513       
Total params: 8,390,081
Trainable params: 8,390,081
Non-trainable params: 0
_________________________________________________________________


In [138]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['mse'])

In [130]:
dataset_train.shape


(8724, 3, 67, 67)

In [150]:
X_train = np.array([item[0] for item in X_train]).reshape(-1,224,224,3)
X_test = np.array([item[0] for item in X_test]).reshape(-1,image_size,image_size,3)
y_train = np.array([item[1] for item in y_train]).reshape(-1)
y_test = np.array([item[1] for item in y_test]).reshape(-1)

ValueError: ignored

In [139]:
model.fit(dataset_train,y_train)

ValueError: ignored